In [6]:
import coiled
from dask.distributed import wait 
import dask.dataframe as dd 
import dask
from dask.utils import format_bytes

### With Arrow strings

In [7]:
dask.config.set({"dataframe.convert-string": True})

In [3]:
%%time

cluster = coiled.Cluster(
    name="lightning-demo-1",
    n_workers=15,
    backend_options={"region": "us-east-2"},
    scheduler_options={"idle_timeout": "2 hours"},
)

Output()

Output()

CPU times: user 5.22 s, sys: 571 ms, total: 5.79 s
Wall time: 1min 55s


In [ ]:
%%time

cluster = coiled.Cluster(name="lightning-demo-1")

In [8]:
%%time

client = cluster.get_client()
client

CPU times: user 174 µs, sys: 8 µs, total: 182 µs
Wall time: 197 µs


<Client: 'tls://10.0.28.241:8786' processes=15 threads=60, memory=223.98 GiB>

In [9]:
%%time

df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={"anon": True},
).persist()

wait(df);

CPU times: user 803 ms, sys: 259 ms, total: 1.06 s
Wall time: 36.2 s


In [52]:
df

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
npartitions=720,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,string,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],int32,int32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,category[unknown],category[unknown],category[unknown],category[unknown],category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
df.dtypes

hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[ns]
on_scene_datetime        datetime64[ns]
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int32
DOLocationID                      int32
trip_miles                      float32
trip_time                         int32
base_passenger_fare             float32
tolls                           float32
bcf                             float32
sales_tax                       float32
congestion_surcharge            float32
airport_fee                     float32
tips                            float32
driver_pay                      float32
shared_request_flag            category
shared_match_flag              category
access_a_ride_flag             category
wav_request_flag               category
wav_match_flag                 category
dtype: object

In [39]:
df.request_datetime.min().compute()

Timestamp('2019-01-31 23:19:44')

In [40]:
df.request_datetime.max().compute()

Timestamp('2023-02-01 00:15:00')

In [41]:
print(f"{df.request_datetime.count().compute():,}")

783,322,943


In [42]:
df.memory_usage(deep=True).compute().sum().apply(format_bytes)

DOLocationID              2.92 GiB
Index                     5.84 GiB
PULocationID              2.92 GiB
access_a_ride_flag      747.29 MiB
airport_fee               2.92 GiB
base_passenger_fare       2.92 GiB
bcf                       2.92 GiB
congestion_surcharge      2.92 GiB
dispatching_base_num      7.31 GiB
driver_pay                2.92 GiB
dropoff_datetime          5.84 GiB
hvfhs_license_num         7.30 GiB
on_scene_datetime         5.84 GiB
originating_base_num      6.18 GiB
pickup_datetime           5.84 GiB
request_datetime          5.84 GiB
sales_tax                 2.92 GiB
shared_match_flag       747.29 MiB
shared_request_flag     747.29 MiB
tips                      2.92 GiB
tolls                     2.92 GiB
trip_miles                2.92 GiB
trip_time                 2.92 GiB
wav_match_flag          747.29 MiB
wav_request_flag        747.29 MiB
dtype: object

In [43]:
client.close()

In [44]:
cluster.close()